# Get Census Data

In [11]:
from census import Census
from us import states
import pandas as pd

# Specify state and county to download (select one)

loc_name, state_codes, county_codes = "balt_city", states.MD.fips, list([510]) # Baltimore
# loc_name, state_codes, county_codes = "greater_balt", states.MD.fips, list([510, 5]) # Baltimore City and County
#loc_name, state_codes, county_codes = "maryland", states.MD.fips, None # All of Maryland
#loc_name, state_codes, county_codes = "delmarva", [states.MD.fips, states.DE.fips, states.VA.fips], None # All of DelMarVa

# CENSUS API Stuff
CENSUS_API = "bdadccb272f83b2244d1b584baef832489fd5c19"
c = Census(CENSUS_API) # Initialize census class with API key

# Generate codes for census variables of interest
var_ids = ["B19001_0{:02d}E".format(x) for x in range(2, 18)] # Household income over 12 months

# Local Storage Parameters
LOCAL_DATA_DIR = './data/'
ATTR_FILE_END = '_census_data.csv'
attr_outfile = LOCAL_DATA_DIR + loc_name + ATTR_FILE_END


## Define helper functions

The following functions are useful for getting the census data and converting it to a pandas dataframe.

In [12]:
def build_bg_fips(record):
    fips_code = record['state'] + record['county'] + record['tract'] + record['block group']
    return str(fips_code)

def build_tract_fips(record):
    fips_code = record['state'] + record['county'] + record['tract']
    return str(fips_code)


def census_bg_to_dataframe(var_list, state_code, county_codes):
    fips_codes = []
    all_records = []
    
    for county in county_codes:        
        census_data = c.acs.get(var_list, {'for': 'tract:*', 'in': 'state:{0}'.format(state_code)})
        
        for idx, record in enumerate(census_data):
            # Build fips codes
            fips_code = build_bg_fips(record)
            census_data[idx]["fips"] = fips_code

            # Eliminate original code components
            key_list = ['state', 'county', 'tract', 'block group']
            for key in key_list:
                if key in census_data[idx]: 
                    del census_data[idx][key]
        
        all_records.extend(census_data)
        
    census_df = pd.DataFrame(all_records)
    census_df = census_df.set_index("fips")
                
    return census_df

def census_tracts_to_dataframe(var_list, state_codes):
    fips_codes = []
    all_records = []
    
    for state_id in state_codes:
        census_data = c.acs.get(var_list, {'for': 'tract:*', 'in': 'state:{0}'.format(state_id)})

        for idx, record in enumerate(census_data):

            # Build fips codes
            fips_code = build_tract_fips(record)
            census_data[idx]["fips"] = fips_code

            # Eliminate original code components
            key_list = ['state', 'county', 'tract']
            for key in key_list:
                if key in census_data[idx]: 
                    del census_data[idx][key]
        
        all_records.extend(census_data)
      
    census_df = pd.DataFrame(all_records)
    census_df = census_df.set_index("fips")
                
    return census_df


In [13]:
# This segment of code will get household income estimates for each block group in Baltimore city
census_df = census_tracts_to_dataframe(var_ids, state_codes)
census_df.to_csv(attr_outfile) # Write to csv